In [5]:
import os
import sys
import ast
from pprint import pprint
import pandas as pd
import matplotlib.pylab as plt
import numpy as np
import itertools as it
import seaborn as sns
from collections import namedtuple
import portion as P
%config InlineBackend.figure_format = 'retina'  # 提高 jupyter notebook 的圖形顯示解析度

def set_data(df, mode='pcap', tz=0):
    if mode == 'pcap':
        df['seq'] = df['seq'].astype('Int32')
        # df['rpkg'] = df['rpkg'].astype('Int8')
        df['frame_id'] = df['frame_id'].astype('Int32')
        df['Timestamp'] = pd.to_datetime(df['Timestamp'])
        df['xmit_time'] = pd.to_datetime(df['xmit_time'])
        df['arr_time'] = pd.to_datetime(df['arr_time'])
        df['Timestamp_epoch'] = df['Timestamp_epoch'].astype('float32')
        df['xmit_time_epoch'] = df['xmit_time_epoch'].astype('float32')
        df['arr_time_epoch'] = df['arr_time_epoch'].astype('float32')
        df['lost'] = df['lost'].astype('boolean')
        df['excl'] = df['excl'].astype('boolean')
        df['latency'] = df['latency'].astype('float32')
    # E-UTRA & NR
    def nr_serv_cel(row):
        pos = row.serv_cel_pos
        if pos == 255:
            return 65535, -160, -50
        else:
            return row[f'PCI{pos}'], row[f'RSRP{pos}'], row[f'RSRQ{pos}']
    if mode == 'lte':
        df = df.rename(columns={
            'RSRP(dBm)': 'RSRP',
            'RSRQ(dB)': 'RSRQ',
            'Serving Cell Index': 'serv_cel_index',
            'Number of Neighbor Cells': 'num_neigh_cels',
            'Number of Detected Cells': 'num_cels',
            }).reindex(['Timestamp','type_id','PCI','RSRP','RSRQ','serv_cel_index','EARFCN','NR_ARFCN',
                        'num_cels','num_neigh_cels','serv_cel_pos','PCI0','RSRP0','RSRQ0',
                        *df.columns.to_list()[df.columns.get_loc('PCI1'):]], axis=1)
        df.loc[df['serv_cel_index'] == '(MI)Unknown', 'serv_cel_index'] = '3_SCell'
        df['num_cels'] = df['num_neigh_cels'] + 1
        ## set dtypes
        df['Timestamp'] = pd.to_datetime(df['Timestamp']) + pd.Timedelta(hours=tz)
        df['type_id'] = df['type_id'].astype('category')
        df['serv_cel_index'] = df['serv_cel_index'].astype('category')
        df['EARFCN'] = df['EARFCN'].astype('Int32')
        df['NR_ARFCN'] = df['NR_ARFCN'].astype('Int32')
        df['num_cels'] = df['num_cels'].astype('UInt8')
        df['num_neigh_cels'] = df['num_neigh_cels'].astype('UInt8')
        df['serv_cel_pos'] = df['serv_cel_pos'].astype('UInt8')
        for tag in df.columns:
            if tag.startswith('PCI'):
                df[tag] = df[tag].astype('Int32')
            if tag.startswith(('RSRP','RSRQ')):
                df[tag] = df[tag].astype('float32')
    if mode == 'nr':
        if df.empty:
            df = pd.DataFrame(columns=['Timestamp','type_id','PCI','RSRP','RSRQ','serv_cel_index','EARFCN','NR_ARFCN',
                                       'num_cels','num_neigh_cels','serv_cel_pos','PCI0','RSRP0','RSRQ0'])
        else:
            df = df.rename(columns={
                'Raster ARFCN': 'NR_ARFCN',
                'Serving Cell Index': 'serv_cel_pos',
                'Num Cells': 'num_cels',
                }).reindex(['Timestamp','type_id','PCI','RSRP','RSRQ','serv_cel_index','EARFCN','NR_ARFCN',
                            'num_cels','num_neigh_cels','serv_cel_pos','PCI0','RSRP0','RSRQ0',
                            *df.columns.to_list()[df.columns.get_loc('PCI1'):]], axis=1)
            df.loc[df['serv_cel_pos'] != 255, 'serv_cel_index'] = 'PSCell'
            df[['PCI','RSRP','RSRQ']] = df.apply(nr_serv_cel, axis=1, result_type='expand')
            df.loc[df['serv_cel_pos'] == 255, 'num_neigh_cels'] = df['num_cels']
            df.loc[df['serv_cel_pos'] != 255, 'num_neigh_cels'] = df['num_cels'] - 1
        ## set dtypes
        df['Timestamp'] = pd.to_datetime(df['Timestamp']) + pd.Timedelta(hours=tz)
        df['type_id'] = df['type_id'].astype('category')
        df['serv_cel_index'] = df['serv_cel_index'].astype('category')
        df['EARFCN'] = df['EARFCN'].astype('Int32')
        df['NR_ARFCN'] = df['NR_ARFCN'].astype('Int32')
        df['num_cels'] = df['num_cels'].astype('UInt8')
        df['num_neigh_cels'] = df['num_neigh_cels'].astype('UInt8')
        df['serv_cel_pos'] = df['serv_cel_pos'].astype('UInt8')
        for tag in df.columns:
            if tag.startswith('PCI'):
                df[tag] = df[tag].astype('Int32')
            if tag.startswith(('RSRP','RSRQ')):
                df[tag] = df[tag].astype('float32')
    return df

In [6]:
# root = '/home/wmnlab/D/database'
root = '/Users/jackbedford/Desktop/MOXA/Code/data'
paths = [s for s in os.listdir(root) if s.startswith('202') and os.path.isdir(os.path.join(root, s))]

md_files = []

for date_dir in paths:
    datedir = os.path.join(root, date_dir)
    md_file = [os.path.join(datedir, s) for s in os.listdir(datedir) if s.endswith('.md')]
    md_files += md_file

md_files = sorted(md_files)

# Select dates
# md_files = md_files[:-1]
# md_files = [md_files[2]]
md_files = [s for s in md_files if '05-04' not in s and '05-07' not in s]

pprint(md_files)

['/Users/jackbedford/Desktop/MOXA/Code/data/2023-03-16/2023-03-16.md',
 '/Users/jackbedford/Desktop/MOXA/Code/data/2023-03-26/2023-03-26.md',
 '/Users/jackbedford/Desktop/MOXA/Code/data/2023-04-01/2023-04-01.md',
 '/Users/jackbedford/Desktop/MOXA/Code/data/2023-04-10/2023-04-10.md',
 '/Users/jackbedford/Desktop/MOXA/Code/data/2023-04-17/2023-04-17.md',
 '/Users/jackbedford/Desktop/MOXA/Code/data/2023-05-15/2023-05-15.md',
 '/Users/jackbedford/Desktop/MOXA/Code/data/2023-05-24/2023-05-24.md',
 '/Users/jackbedford/Desktop/MOXA/Code/data/2023-05-26/2023-05-26.md']


In [7]:
class Experiment():
    
    def __init__(self, dir_name, setting):
        
        self.dir_name = dir_name
        self.setting = setting
        
    def __repr__(self):
        return f'{self.dir_name} -> {self.setting}'

exp_list = []

for md_file in md_files:
    
    md_f = open(md_file, 'r')
    lines = []
    l = md_f.readline()[:-1]
    
    while l:
        
        if l == '#endif':
            break
        lines.append(l)
        l = md_f.readline()[:-1]
        
    md_f.close()
    
    for exp, dic in zip(lines[::2], lines[1::2]):
        
        dictionary = ast.literal_eval(dic)
        parent_dir = os.path.dirname(md_file)
        experiment = Experiment(dir_name = os.path.join(parent_dir, exp), setting=dictionary)
        exp_list.append(experiment)
        
pprint(exp_list)

[/Users/jackbedford/Desktop/MOXA/Code/data/2023-03-16/Bandlock_Udp_B1_B3_B7_B8_RM500Q -> {'qc00': 'B1', 'qc02': 'B7', 'qc03': 'B8'},
 /Users/jackbedford/Desktop/MOXA/Code/data/2023-03-26/Bandlock_Udp_All_RM500Q -> {'qc00': 'B3', 'qc02': 'B7', 'qc03': 'B8'},
 /Users/jackbedford/Desktop/MOXA/Code/data/2023-04-01/Bandlock_Udp_All_LTE_B1B3_B1B8_RM500Q -> {'qc00': 'B1B3', 'qc01': 'B1B8', 'qc02': 'LTE', 'qc03': 'All'},
 /Users/jackbedford/Desktop/MOXA/Code/data/2023-04-01/Bandlock_Udp_B1_B3_B7_B8_RM500Q -> {'qc00': 'B1', 'qc01': 'B3', 'qc02': 'B7', 'qc03': 'B8'},
 /Users/jackbedford/Desktop/MOXA/Code/data/2023-04-10/Experiment1 -> {'qc00': 'All', 'qc01': 'All', 'qc02': 'All', 'qc03': 'All'},
 /Users/jackbedford/Desktop/MOXA/Code/data/2023-04-10/Experiment2 -> {'qc00': 'B1', 'qc01': 'B3', 'qc02': 'B7', 'qc03': 'B8'},
 /Users/jackbedford/Desktop/MOXA/Code/data/2023-04-10/Experiment3 -> {'qc00': 'LTE', 'qc01': 'All', 'qc02': 'B7B8', 'qc03': 'B7'},
 /Users/jackbedford/Desktop/MOXA/Code/data/2023

In [8]:
# Set band parameters
_schms = ['All','B3B7B8','LTE','B1','B3','B7','B8','B1B3','B1B7','B1B8','B3B7','B3B8','B7B8']
schms = [f'{_schms[i]}+{_schms[j]}' for i in range(len(_schms)) for j in range(i, len(_schms))]
# schms = ['B1+B3','B1+B7','B3+B8']

rrc_dict = {}
nr_dict = {}
lte_dict = {}
ul_pkt_dict = {}
dl_pkt_dict = {}

for schm in schms:
        
    path_list = []
    
    for exp in exp_list:
        
        reverse_dict = {v:k for k, v in exp.setting.items()}
        sch1 = schm.split('+')[0]
        sch2 = schm.split('+')[1]
        # print(sch1, sch2)
        
        try:
            dev1 = reverse_dict[sch1]
            dev2 = reverse_dict[sch2]
        except:
            continue
        
        device_dir1 = os.path.join(exp.dir_name, dev1)
        device_dir2 = os.path.join(exp.dir_name, dev2)
        
        for trace1, trace2 in zip(os.listdir(device_dir1), os.listdir(device_dir2)):
        
            if '#' not in trace1 or '#' not in trace2:
                continue
        
            trace_dir1 = os.path.join(device_dir1, trace1, 'data')
            trace_dir2 = os.path.join(device_dir2, trace2, 'data')
            path_list.append((trace_dir1, trace_dir2))

    path_list = sorted(path_list)

    rrc_list, nr_list, lte_list, ul_pkt_list, dl_pkt_list = [], [], [], [], []

    for (path1, path2) in path_list:
        
        # rrc_list += [(os.path.join(path1, s1), os.path.join(path2, s2)) for s1 in os.listdir(path1) for s2 in os.listdir(path2) if (s1.endswith('_rrc.pkl') and s2.endswith('_rrc.pkl'))]
        rrc_list += [(os.path.join(path1, 'handover_info_list.pkl'), os.path.join(path2, 'handover_info_list.pkl'))]
        nr_list += [(os.path.join(path1, s1), os.path.join(path2, s2)) for s1 in os.listdir(path1) for s2 in os.listdir(path2) if (s1.endswith('_nr_ml1.pkl') and s2.endswith('_nr_ml1.pkl'))]
        lte_list += [(os.path.join(path1, s1), os.path.join(path2, s2)) for s1, s2 in zip(os.listdir(path1), os.listdir(path2)) if (s1.endswith('_ml1.pkl') and not s1.endswith('_nr_ml1.pkl') and s2.endswith('_ml1.pkl') and not s2.endswith('_nr_ml1.pkl'))]
        ul_pkt_list += [(os.path.join(path1, 'udp_uplk_loss_latency.pkl'), os.path.join(path2, 'udp_uplk_loss_latency.pkl'))]
        dl_pkt_list += [(os.path.join(path1, 'udp_dnlk_loss_latency.pkl'), os.path.join(path2, 'udp_dnlk_loss_latency.pkl'))]

    rrc_dict[schm] = rrc_list
    nr_dict[schm] = nr_list
    lte_dict[schm] = lte_list
    ul_pkt_dict[schm] = ul_pkt_list
    dl_pkt_dict[schm] = dl_pkt_list

In [9]:
schms.remove('All+B3B7B8')
del rrc_dict['All+B3B7B8']
del nr_dict['All+B3B7B8']
del lte_dict['All+B3B7B8']
del ul_pkt_dict['All+B3B7B8']
del dl_pkt_dict['All+B3B7B8']

remove_list = []
for schm in schms:
    if 'B3B7B8' not in schm:
        continue
    # print(schm)
    schm1, _schm1 = schm.split('+')[0], schm.split('+')[0]
    schm2, _schm2 = schm.split('+')[1], schm.split('+')[1]
    if schm1 == 'B3B7B8': _schm1 = 'All'
    if schm2 == 'B3B7B8': _schm2 = 'All'
    # print(schm1, _schm1)
    # print(schm2, _schm2)
    rrc_dict[f'{_schm1}+{_schm2}'] = rrc_dict[f'{_schm1}+{_schm2}'] + rrc_dict[f'{schm1}+{schm2}']
    nr_dict[f'{_schm1}+{_schm2}'] = nr_dict[f'{_schm1}+{_schm2}'] + nr_dict[f'{schm1}+{schm2}']
    lte_dict[f'{_schm1}+{_schm2}'] = lte_dict[f'{_schm1}+{_schm2}'] + lte_dict[f'{schm1}+{schm2}']
    ul_pkt_dict[f'{_schm1}+{_schm2}'] = ul_pkt_dict[f'{_schm1}+{_schm2}'] + ul_pkt_dict[f'{schm1}+{schm2}']
    dl_pkt_dict[f'{_schm1}+{_schm2}'] = dl_pkt_dict[f'{_schm1}+{_schm2}'] + dl_pkt_dict[f'{schm1}+{schm2}']
    
    remove_list.append(f'{schm1}+{schm2}')

for tag in remove_list:
    schms.remove(tag)
    del rrc_dict[tag]
    del nr_dict[tag]
    del lte_dict[tag]
    del ul_pkt_dict[tag]
    del dl_pkt_dict[tag]

In [10]:
remove_list = []

for schm in schms:
    schm1 = schm.split('+')[0]
    schm2 = schm.split('+')[1]
    if schm1 != schm2: 
        continue
    remove_list.append(f'{schm1}+{schm2}')

for tag in remove_list:
    schms.remove(tag)
    del rrc_dict[tag]
    del nr_dict[tag]
    del lte_dict[tag]
    del ul_pkt_dict[tag]
    del dl_pkt_dict[tag]

In [11]:
print(schms)
# pprint(list(rrc_dict.keys()))
# pprint(rrc_dict['All+All'])

['All+LTE', 'All+B1', 'All+B3', 'All+B7', 'All+B8', 'All+B1B3', 'All+B1B7', 'All+B1B8', 'All+B3B7', 'All+B3B8', 'All+B7B8', 'LTE+B1', 'LTE+B3', 'LTE+B7', 'LTE+B8', 'LTE+B1B3', 'LTE+B1B7', 'LTE+B1B8', 'LTE+B3B7', 'LTE+B3B8', 'LTE+B7B8', 'B1+B3', 'B1+B7', 'B1+B8', 'B1+B1B3', 'B1+B1B7', 'B1+B1B8', 'B1+B3B7', 'B1+B3B8', 'B1+B7B8', 'B3+B7', 'B3+B8', 'B3+B1B3', 'B3+B1B7', 'B3+B1B8', 'B3+B3B7', 'B3+B3B8', 'B3+B7B8', 'B7+B8', 'B7+B1B3', 'B7+B1B7', 'B7+B1B8', 'B7+B3B7', 'B7+B3B8', 'B7+B7B8', 'B8+B1B3', 'B8+B1B7', 'B8+B1B8', 'B8+B3B7', 'B8+B3B8', 'B8+B7B8', 'B1B3+B1B7', 'B1B3+B1B8', 'B1B3+B3B7', 'B1B3+B3B8', 'B1B3+B7B8', 'B1B7+B1B8', 'B1B7+B3B7', 'B1B7+B3B8', 'B1B7+B7B8', 'B1B8+B3B7', 'B1B8+B3B8', 'B1B8+B7B8', 'B3B7+B3B8', 'B3B7+B7B8', 'B3B8+B7B8']


## Label Handover

In [12]:
## Downlink
dl_ImpactScope = {
    'LTE_HO': (1.4, 0.75),
    'MN_HO': (2.8, 2.2),
    'SN_HO': (6, 6.3),
    'SN_Setup': (0.5, 2),
    'SN_Rel': (0.4, 0.2),
    'SN_Rel_MN_HO': (0.6, 1),
    'SCG_Failure_t310-Expiry (0)': (3.2, 1.2),
    'SCG_Failure_randomAccessProblem (1)': (2, 2),
    'SCG_Failure_rlc-MaxNumRetx (2)': (2, 2),
    'SCG_Failure_synchReconfigFailureSCG (3)': (0.1, 0.1),
    'MCG_Failure_reconfigurationFailure (0)': (2, 2),
    'MCG_Failure_handoverFailure (1)': (0.75, 1.25),
    'MCG_Failure_otherFailure (2)': (5.6, 1.7),
    'NAS_Recovery_reconfigurationFailure (0)': (2, 2),
    'NAS_Recovery_handoverFailure (1)': (0.65, 0.7),
    'NAS_Recovery_otherFailure (2)': (3.6, 1.85),
}

## Uplink
ul_ImpactScope = {
    'LTE_HO': (2, 2),
    'MN_HO': (1, 1.7),
    'SN_HO': (2, 2),
    'SN_Setup': (0.5, 2),
    'SN_Rel': (0.2, 1.5),
    'SN_Rel_MN_HO': (1.2 , 0.85),
    'SCG_Failure_t310-Expiry (0)': (2, 1.6),
    'SCG_Failure_randomAccessProblem (1)': (0.1, 0.1),
    'SCG_Failure_rlc-MaxNumRetx (2)': (1.5, 0.1),
    'SCG_Failure_synchReconfigFailureSCG (3)': (0.1, 0.1),
    'MCG_Failure_reconfigurationFailure (0)': (2, 2),
    'MCG_Failure_handoverFailure (1)': (0.65, 0.15),
    'MCG_Failure_otherFailure (2)': (5, 1.5),
    'NAS_Recovery_reconfigurationFailure (0)': (2, 2),
    'NAS_Recovery_handoverFailure (1)': (0.1, 0.4),
    'NAS_Recovery_otherFailure (2)': (1.65, 1.7),
}

In [15]:
sorter = ['LTE_HO','MN_HO','SN_HO','MNSN_HO','SN_Rel','SN_Setup','SN_Rel_MN_HO','SN_Setup_MN_HO',
          'SCG_Failure_t310-Expiry (0)','SCG_Failure_randomAccessProblem (1)','SCG_Failure_rlc-MaxNumRetx (2)','SCG_Failure_synchReconfigFailureSCG (3)',
          'SCG_Failure_scg-ReconfigFailure (4)','SCG_Failure_srb3-IntegrityFailure (5)','SCG_Failure_other-r16 (6)',
          'MCG_Failure_reconfigurationFailure (0)','MCG_Failure_handoverFailure (1)','MCG_Failure_otherFailure (2)',
          'NAS_Recovery_reconfigurationFailure (0)','NAS_Recovery_handoverFailure (1)','NAS_Recovery_otherFailure (2)']

def cut_head_tail(df_HO, df, mode='ul'):
    if mode == 'ul':
        start = df.iloc[0].xmit_time
        stop = df.iloc[-1].xmit_time
        df_HO = df_HO.query('Timestamp >= @start & Timestamp <= @stop').copy().reset_index(drop=True)
    if mode == 'dl':
        start = df.iloc[0].arr_time
        stop = df.iloc[-1].arr_time
        df_HO = df_HO.query('Timestamp >= @start & Timestamp <= @stop').copy().reset_index(drop=True)
    return df_HO

def is_disjoint(set1, set2):
    """
    Check if two sets are disjoint.
    """
    return (set1 & set2).empty

def is_disjoint_dict(E):
    test_intv = P.empty()
    for key, val in E.items():
        # print(key)
        for intv in val:
            if is_disjoint(test_intv, intv.interval):
                test_intv = test_intv | intv.interval
            else:
                print(key, intv.index)
                return False
    return True

def interp(x, y, ratio):
    """
    Interpolation

    Args:
        x, y (datetime.datetime): x < y
        ratio (float): a decimal numeral in a range [0, 1]; 0 means break at x, 1 means break at y.
    Returns:
        (datetime.datetime): breakpoint of interpolation
    """
    return x + (y - x) * ratio

def get_ho_interval(df, sec=(1, 3), ratio=0.5,
                 ignored=['Conn_Setup','Conn_Rel'],
                 handover=['LTE_HO','MN_HO','SN_HO','SN_Setup','SN_Rel','SN_Rel_MN_HO'],
                 linkfailure=['SCG_Failure_t310-Expiry (0)','SCG_Failure_randomAccessProblem (1)','SCG_Failure_rlc-MaxNumRetx (2)','SCG_Failure_synchReconfigFailureSCG (3)',
                              'SCG_Failure_scg-ReconfigFailure (4)','SCG_Failure_srb3-IntegrityFailure (5)','SCG_Failure_other-r16 (6)',
                              'MCG_Failure_reconfigurationFailure (0)','MCG_Failure_handoverFailure (1)','MCG_Failure_otherFailure (2)',
                              'NAS_Recovery_reconfigurationFailure (0)','NAS_Recovery_handoverFailure (1)','NAS_Recovery_otherFailure (2)'],
                 ImpactScope=dl_ImpactScope):
    
    HO_INTV = namedtuple('HO_INTV', 'index, interval, state1, state2, st_scel, cause, interrupt, \
                                     ePCI, earfcn, nrPCI, ePCI1, earfcn1, nrPCI1', defaults=tuple([None]*12))
    
    def ignore_col(row):
        if row.ho_type in ignored:
            return False
        else:
            return True
    df = df[df.apply(ignore_col, axis=1)].reset_index(drop=True)
    
    column_names = []
    for type_name in handover + linkfailure:
        column_names += ["before_{}".format(type_name), "during_{}".format(type_name), "after_{}".format(type_name)]
    E = { col:[] for col in column_names }
    
    for i, row in df.iterrows():
        prior_row = df.iloc[i-1] if i != 0 else None
        post_row = df.iloc[i+1] if i != len(df)-1 else None
        ### peek the next event
        if i != len(df)-1 and pd.notna(row.end) and row.end > post_row.start:
            print(i, row.start, row.end, row.ho_type, row.cause)
            print(i+1, post_row.start, post_row.end, post_row.ho_type, post_row.cause)
            continue
        ### peri_interval
        if pd.isna(row.end):
            peri_interval = P.singleton(row.start)
        else:
            peri_interval = P.closed(row.start, row.end)
        ### prior_interval
        type_name = row.ho_type if row.ho_type not in ['SCG_Failure','MCG_Failure','NAS_Recovery'] else row.ho_type + '_' + row.cause
        C = row.start - pd.Timedelta(seconds=ImpactScope[type_name][0])
        D = row.start
        prior_interval = P.closedopen(C, D)
        if ratio != None and i != 0:
            type_name = prior_row.ho_type if prior_row.ho_type not in ['SCG_Failure','MCG_Failure','NAS_Recovery'] else prior_row.ho_type + '_' + prior_row.cause
            A = max(prior_row.start, prior_row.end)
            B = max(prior_row.start, prior_row.end) + pd.Timedelta(seconds=ImpactScope[type_name][1])
            if P.openclosed(A, B).overlaps(prior_interval):
                # print("Overlaps with the previous!")
                bkp = interp(C, B, ratio)
                bkp = max(bkp, A)  # to avoid the breakpoint overlaps the previous event's duration
                # bkp = min(max(bkp, A), D)  # 我不侵犯到其他任何人，代表其他人也不會侵犯到我！
                prior_interval = P.closedopen(bkp, D)
                if A in prior_interval:
                    prior_interval = P.open(bkp, D)
                # blindly set as open inverval is fine, but may miss one point.
        ### post_interval
        type_name = row.ho_type if row.ho_type not in ['SCG_Failure','MCG_Failure','NAS_Recovery'] else row.ho_type + '_' + row.cause
        C = row.end
        D = row.end + pd.Timedelta(seconds=ImpactScope[type_name][1])
        post_interval = P.openclosed(C, D)
        if ratio != None and i != len(df)-1:
            type_name = post_row.ho_type if post_row.ho_type not in ['SCG_Failure','MCG_Failure','NAS_Recovery'] else post_row.ho_type + '_' + post_row.cause
            A = min(post_row.start, post_row.end) - pd.Timedelta(seconds=ImpactScope[type_name][0])
            B = min(post_row.start, post_row.end)
            if P.closedopen(A, B).overlaps(post_interval):
                # print("Overlaps with the following!")
                bkp = interp(A, D, ratio)
                bkp = min(bkp, B)  # to avoid the breakpoint overlaps the following event's duration
                # bkp = max(min(bkp, B), C)  # 我不侵犯到其他任何人，代表其他人也不會侵犯到我！
                post_interval = P.open(C, bkp)
        ### append dictionary
        type_name = row.ho_type if row.ho_type not in ['SCG_Failure','MCG_Failure','NAS_Recovery'] else row.ho_type + '_' + row.cause
        state1, state2 = 'sn_change', 'sn_change'
        if type_name in linkfailure:
            state1, state2 = 'link_failure', 'link_failure'
        if type_name in ['LTE_HO','MN_HO','SN_Rel_MN_HO']:
            state1 = 'inter_freq' if row.sFreq != row.tFreq else 'intra_freq'
            if pd.notna(row.eNB) and pd.notna(row.eNB1) and row.eNB != row.eNB1:
                state2 = 'inter_enb'
            elif row.sPCI != row.tPCI:
                state2 = 'inter_sector'
            elif row.sPCI == row.tPCI:
                state2 = 'intra_sector'
            else:
                print("************** inter_enb, unknown eNB_ID **************")
                state2 = 'inter_enb'
        E[f'before_{type_name}'].append(HO_INTV(i, prior_interval, state1, state2, row.st_scel, row.cause, row.intr, row.sPCI, row.sFreq, row.snrPCI, row.tPCI, row.tFreq, row.tnrPCI))
        E[f'during_{type_name}'].append(HO_INTV(i, peri_interval, state1, state2, row.st_scel, row.cause, row.intr, row.sPCI, row.sFreq, row.snrPCI, row.tPCI, row.tFreq, row.tnrPCI))
        E[f'after_{type_name}'].append(HO_INTV(i, post_interval, state1, state2, row.st_scel, row.cause, row.intr, row.sPCI, row.sFreq, row.snrPCI, row.tPCI, row.tFreq, row.tnrPCI))
        ### check whether the intervals are pairwise disjoint
        if not is_disjoint_dict(E):
            print('Warning: Intervals are not totally disjoint!')
    return E

def label_ho_info(df, E, mode='ul'):
    def removeprefix(string, prefix=['before','during','after']):
        for pref in prefix:
            if string.startswith(pref):
                return pref, string[len(pref)+1:]
        return None, string
    
    df = df.reindex(columns=[*list(df.columns),
            'ho_index','ho_stage','ho_type','ho_type1','ho_type2','ho_scel','ho_cause','ho_intr',
            'ho_ePCI','ho_earfcn','ho_nrPCI','ho_ePCI1','ho_earfcn1','ho_nrPCI1'])
    
    df[['ho_index','ho_stage','ho_type','ho_type1','ho_type2','ho_scel','ho_cause','ho_intr',
        'ho_ePCI','ho_earfcn','ho_nrPCI','ho_ePCI1','ho_earfcn1','ho_nrPCI1']] = \
        [-1, '-', 'stable', 'stable', 'stable', 0, None, None,
         None, None, None, None, None, None]
            
    for key, val in E.items():
        pref, key = removeprefix(key)
        for intv in val:
            if intv.interval.empty:
                continue
            # print(pref, key)
            # print(intv.interval)
            if mode == 'ul':
                df.loc[(df['xmit_time'] >= intv.interval.lower) & (df['xmit_time'] <= intv.interval.upper),
                       ('ho_index','ho_stage','ho_type','ho_type1','ho_type2','ho_scel','ho_cause','ho_intr',
                        'ho_ePCI','ho_earfcn','ho_nrPCI','ho_ePCI1','ho_earfcn1','ho_nrPCI1')] = \
                        [intv.index, pref, key, intv.state1, intv.state2, intv.st_scel, intv.cause, intv.interrupt,
                        intv.ePCI, intv.earfcn, intv.nrPCI, intv.ePCI1, intv.earfcn1, intv.nrPCI1]
            if mode == 'dl':
                df.loc[(df['arr_time'] >= intv.interval.lower) & (df['arr_time'] <= intv.interval.upper),
                       ('ho_index','ho_stage','ho_type','ho_type1','ho_type2','ho_scel','ho_cause','ho_intr',
                        'ho_ePCI','ho_earfcn','ho_nrPCI','ho_ePCI1','ho_earfcn1','ho_nrPCI1')] = \
                        [intv.index, pref, key, intv.state1, intv.state2, intv.st_scel, intv.cause, intv.interrupt,
                        intv.ePCI, intv.earfcn, intv.nrPCI, intv.ePCI1, intv.earfcn1, intv.nrPCI1]
    
    df['ho_type0'] = df['ho_type']
    df.loc[np.in1d(df['ho_type'], ['SCG_Failure','MCG_Failure','NAS_Recovery']), 'ho_type0'] = \
        df.loc[np.in1d(df['ho_type'], ['SCG_Failure','MCG_Failure','NAS_Recovery']), 'ho_type'] + '_' + df.loc[np.in1d(df['ho_type'], ['SCG_Failure','MCG_Failure','NAS_Recovery']), 'ho_cause']
    
    df['_ho_type'] = df['ho_type']
    df['_ho_type0'] = df['ho_type0']
    df['_ho_type1'] = df['ho_type1']
    df['_ho_type2'] = df['ho_type2']
    df.loc[~np.in1d(df['ho_type'], ['stable']), '_ho_type'] = df.loc[~np.in1d(df['ho_type'], ['stable']), 'ho_stage'] + '_' + df.loc[~np.in1d(df['ho_type'], ['stable']), 'ho_type']
    df.loc[~np.in1d(df['ho_type'], ['stable']), '_ho_type0'] = df.loc[~np.in1d(df['ho_type'], ['stable']), 'ho_stage'] + '_' + df.loc[~np.in1d(df['ho_type'], ['stable']), 'ho_type0']
    df.loc[~np.in1d(df['ho_type'], ['stable']), '_ho_type1'] = df.loc[~np.in1d(df['ho_type'], ['stable']), 'ho_stage'] + '_' + df.loc[~np.in1d(df['ho_type'], ['stable']), 'ho_type1']
    df.loc[~np.in1d(df['ho_type'], ['stable']), '_ho_type2'] = df.loc[~np.in1d(df['ho_type'], ['stable']), 'ho_stage'] + '_' + df.loc[~np.in1d(df['ho_type'], ['stable']), 'ho_type2']
    
    df['ho_index'] = df['ho_index'].astype('Int32')
    df['ho_stage'] = df['ho_stage'].astype('category')
    df['ho_type'] = df['ho_type'].astype('category')
    df['ho_type0'] = df['ho_type0'].astype('category')
    df['ho_type1'] = df['ho_type1'].astype('category')
    df['ho_type2'] = df['ho_type2'].astype('category')
    df['ho_scel'] = df['ho_scel'].astype('Int8')
    df['ho_cause'] = df['ho_cause'].astype('category')
    df['ho_intr'] = df['ho_intr'].astype('float32')
    for tag in df.columns[df.columns.get_loc('ho_ePCI'):df.columns.get_loc('ho_nrPCI1')+1]:
        df[tag] = df[tag].astype('Int32')
    df['_ho_type'] = df['_ho_type'].astype('category')
    df['_ho_type0'] = df['_ho_type'].astype('category')
    df['_ho_type1'] = df['_ho_type1'].astype('category')
    df['_ho_type2'] = df['_ho_type2'].astype('category')
    
    return df

### Stage1: Read & Label

In [16]:
for schm in schms:
    # print(schm)
    # pprint(rrc_dict[schm])
    for (dl1, dl2), (ul1, ul2), (ho1, ho2) in zip(dl_pkt_dict[schm], ul_pkt_dict[schm], rrc_dict[schm]):
        ### Read dl
        # print(dl1)
        # print(dl2)
        
        df_dl1 = set_data(pd.read_pickle(dl1))
        df_dl2 = set_data(pd.read_pickle(dl2))
        st = max([df_dl1['seq'].array[0], df_dl2['seq'].array[0]])
        et = min([df_dl1['seq'].array[-1], df_dl2['seq'].array[-1]])
        df_dl1 = df_dl1[(df_dl1['seq'] >= st) & (df_dl1['seq'] <= et)].reset_index(drop=True)
        df_dl2 = df_dl2[(df_dl2['seq'] >= st) & (df_dl2['seq'] <= et)].reset_index(drop=True)
        
        # df_dl = pd.merge(df_dl1.copy(), df_dl2.copy(), on=['seq'], suffixes=('_m','_s')).copy()

        ### Read ul
        # print(ul1)
        # print(ul2)
        
        df_ul1 = set_data(pd.read_pickle(ul1))
        df_ul2 = set_data(pd.read_pickle(ul2))
        st = max([df_ul1['seq'].array[0], df_ul2['seq'].array[0]])
        et = min([df_ul1['seq'].array[-1], df_ul2['seq'].array[-1]])
        df_ul1 = df_ul1[(df_ul1['seq'] >= st) & (df_ul1['seq'] <= et)].reset_index(drop=True)
        df_ul2 = df_ul2[(df_ul2['seq'] >= st) & (df_ul2['seq'] <= et)].reset_index(drop=True)
        
        # df_ul = pd.merge(df_ul1.copy(), df_ul2.copy(), on=['seq'], suffixes=('_m','_s')).copy()
    
        ### Read ho & label
        # print(ho1)
        # print(ho2)
        
        df_ho1 = pd.read_pickle(ho1)
        df_ho2 = pd.read_pickle(ho2)
        
        E_ul = get_ho_interval(df_ho1.copy(), ImpactScope=ul_ImpactScope)
        E_dl = get_ho_interval(df_ho1.copy(), ImpactScope=dl_ImpactScope)
        df_ul1 = label_ho_info(df_ul1.copy(), E_ul, mode='ul')
        df_dl1 = label_ho_info(df_dl1.copy(), E_dl, mode='dl')
        print(f'{ul1[:-4]}_ho.pkl')
        print(f'{dl1[:-4]}_ho.pkl')
        df_ul1.to_pickle(f'{ul1[:-4]}_ho.pkl')
        df_dl1.to_pickle(f'{dl1[:-4]}_ho.pkl')

        E_ul = get_ho_interval(df_ho2.copy(), ImpactScope=ul_ImpactScope)
        E_dl = get_ho_interval(df_ho2.copy(), ImpactScope=dl_ImpactScope)
        df_ul2 = label_ho_info(df_ul2.copy(), E_ul, mode='ul')
        df_dl2 = label_ho_info(df_dl2.copy(), E_dl, mode='dl')
        print(f'{ul2[:-4]}_ho.pkl')
        print(f'{dl2[:-4]}_ho.pkl')
        df_ul2.to_pickle(f'{ul2[:-4]}_ho.pkl')
        df_dl2.to_pickle(f'{dl2[:-4]}_ho.pkl')
    #     break
    # break

/Users/jackbedford/Desktop/MOXA/Code/data/2023-04-01/Bandlock_Udp_All_LTE_B1B3_B1B8_RM500Q/qc03/#02/data/udp_uplk_loss_latency_ho.pkl
/Users/jackbedford/Desktop/MOXA/Code/data/2023-04-01/Bandlock_Udp_All_LTE_B1B3_B1B8_RM500Q/qc03/#02/data/udp_dnlk_loss_latency_ho.pkl
/Users/jackbedford/Desktop/MOXA/Code/data/2023-04-01/Bandlock_Udp_All_LTE_B1B3_B1B8_RM500Q/qc02/#01/data/udp_uplk_loss_latency_ho.pkl
/Users/jackbedford/Desktop/MOXA/Code/data/2023-04-01/Bandlock_Udp_All_LTE_B1B3_B1B8_RM500Q/qc02/#01/data/udp_dnlk_loss_latency_ho.pkl
/Users/jackbedford/Desktop/MOXA/Code/data/2023-04-01/Bandlock_Udp_All_LTE_B1B3_B1B8_RM500Q/qc03/#03/data/udp_uplk_loss_latency_ho.pkl
/Users/jackbedford/Desktop/MOXA/Code/data/2023-04-01/Bandlock_Udp_All_LTE_B1B3_B1B8_RM500Q/qc03/#03/data/udp_dnlk_loss_latency_ho.pkl
/Users/jackbedford/Desktop/MOXA/Code/data/2023-04-01/Bandlock_Udp_All_LTE_B1B3_B1B8_RM500Q/qc02/#02/data/udp_uplk_loss_latency_ho.pkl
/Users/jackbedford/Desktop/MOXA/Code/data/2023-04-01/Bandlock_

### Stage2: Combine

In [17]:
sorter = ['stable',
          'LTE_HO','MN_HO','SN_HO','MNSN_HO','SN_Rel','SN_Setup','SN_Rel_MN_HO','SN_Setup_MN_HO',
          'SCG_Failure_t310-Expiry (0)','SCG_Failure_randomAccessProblem (1)','SCG_Failure_rlc-MaxNumRetx (2)','SCG_Failure_synchReconfigFailureSCG (3)',
          'SCG_Failure_scg-ReconfigFailure (4)','SCG_Failure_srb3-IntegrityFailure (5)','SCG_Failure_other-r16 (6)',
          'MCG_Failure_reconfigurationFailure (0)','MCG_Failure_handoverFailure (1)','MCG_Failure_otherFailure (2)',
          'NAS_Recovery_reconfigurationFailure (0)','NAS_Recovery_handoverFailure (1)','NAS_Recovery_otherFailure (2)']

# sorter1 = list(it.permutations(sorter, 2))
sorter1 = list(it.product(sorter, repeat=2))
sorter1 = ['+'.join([s[0],s[1]]) for s in sorter1]
print(len(sorter1))
print(sorter1)
sorter2 = [s.split('+')[0] for s in sorter1]
sorter3 = [s.split('+')[1] for s in sorter1]

table = pd.DataFrame(columns=['ho_count',
                              'ul_lost_m','ul_lost_ms','ul_lost_mlss',
                              'ul_excl_m','ul_excl_ms','ul_excl_mess',
                              'ul_sent','ul_recv_m','ul_recv_ms',
                              'dl_lost_m','dl_lost_ms','dl_lost_mlss',
                              'dl_excl_m','dl_excl_ms','dl_excl_mess',
                              'dl_sent','dl_recv_m','dl_recv_ms'],
                     index=[sorter2, sorter3, sorter1]).fillna(0).reset_index() \
                    .rename(columns={'level_0':'ho_type0_m', 'level_1':'ho_type0_s', 'level_2':'ho_type0_ms'})

display(table)

484
['stable+stable', 'stable+LTE_HO', 'stable+MN_HO', 'stable+SN_HO', 'stable+MNSN_HO', 'stable+SN_Rel', 'stable+SN_Setup', 'stable+SN_Rel_MN_HO', 'stable+SN_Setup_MN_HO', 'stable+SCG_Failure_t310-Expiry (0)', 'stable+SCG_Failure_randomAccessProblem (1)', 'stable+SCG_Failure_rlc-MaxNumRetx (2)', 'stable+SCG_Failure_synchReconfigFailureSCG (3)', 'stable+SCG_Failure_scg-ReconfigFailure (4)', 'stable+SCG_Failure_srb3-IntegrityFailure (5)', 'stable+SCG_Failure_other-r16 (6)', 'stable+MCG_Failure_reconfigurationFailure (0)', 'stable+MCG_Failure_handoverFailure (1)', 'stable+MCG_Failure_otherFailure (2)', 'stable+NAS_Recovery_reconfigurationFailure (0)', 'stable+NAS_Recovery_handoverFailure (1)', 'stable+NAS_Recovery_otherFailure (2)', 'LTE_HO+stable', 'LTE_HO+LTE_HO', 'LTE_HO+MN_HO', 'LTE_HO+SN_HO', 'LTE_HO+MNSN_HO', 'LTE_HO+SN_Rel', 'LTE_HO+SN_Setup', 'LTE_HO+SN_Rel_MN_HO', 'LTE_HO+SN_Setup_MN_HO', 'LTE_HO+SCG_Failure_t310-Expiry (0)', 'LTE_HO+SCG_Failure_randomAccessProblem (1)', 'LTE_HO

,ho_type0_m,ho_type0_s,ho_type0_ms,ho_count,ul_lost_m,ul_lost_ms,ul_lost_mlss,ul_excl_m,ul_excl_ms,ul_excl_mess,...,ul_recv_ms,dl_lost_m,dl_lost_ms,dl_lost_mlss,dl_excl_m,dl_excl_ms,dl_excl_mess,dl_sent,dl_recv_m,dl_recv_ms
0,stable,stable,stable+stable,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,stable,LTE_HO,stable+LTE_HO,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,stable,MN_HO,stable+MN_HO,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,stable,SN_HO,stable+SN_HO,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,stable,MNSN_HO,stable+MNSN_HO,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
479,NAS_Recovery_otherFailure (2),MCG_Failure_handoverFailure (1),NAS_Recovery_otherFailure (2)+MCG_Failure_hand...,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
480,NAS_Recovery_otherFailure (2),MCG_Failure_otherFailure (2),NAS_Recovery_otherFailure (2)+MCG_Failure_othe...,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
481,NAS_Recovery_otherFailure (2),NAS_Recovery_reconfigurationFailure (0),NAS_Recovery_otherFailure (2)+NAS_Recovery_rec...,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
482,NAS_Recovery_otherFailure (2),NAS_Recovery_handoverFailure (1),NAS_Recovery_otherFailure (2)+NAS_Recovery_han...,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [18]:
def dual_radio_stats(df_main, df_subr):
    dfs = [pd.DataFrame(), pd.DataFrame()]
    
    ### Uplink
    if df_main[0].empty:
        table = pd.DataFrame(columns=['ho_type0_ms','ho_count',
                                      'ul_lost_m','ul_lost_ms','ul_lost_mlss',
                                      'ul_excl_m','ul_excl_ms','ul_excl_mess',
                                      'ul_sent','ul_recv_m','ul_recv_ms'])
    else:
        dfs[0] = pd.merge(df_main[0], df_subr[0], on=['seq'], suffixes=('_m','_s')).copy()
        # dfs[0]['ho_type0_m'] = dfs[0]['ho_type0_m'].cat.set_categories(sorter)
        # dfs[0]['ho_type0_s'] = dfs[0]['ho_type0_s'].cat.set_categories(sorter)
        dfs[0]['ho_type0_ms'] = dfs[0]['ho_type0_m'].astype('string') + '+' + dfs[0]['ho_type0_s'].astype('string')
        dfs[0]['ho_type0_ms'] = dfs[0]['ho_type0_ms'].astype('category')
        dfs[0]['ho_type0_ms'] = dfs[0]['ho_type0_ms'].cat.set_categories(sorter1)
        ## main ul lost
        table = dfs[0].loc[dfs[0]['lost_m'], ['ho_type0_ms']].value_counts()
        table = pd.DataFrame(table).reset_index().rename(columns={0:f'ul_lost_m'})
        ## system ul lost
        table1 = dfs[0].loc[dfs[0]['lost_m'] & dfs[0]['lost_s'], ['ho_type0_ms']].value_counts()
        table1 = pd.DataFrame(table1).reset_index().rename(columns={0:f'ul_lost_ms'})
        table = table.merge(table1, on=['ho_type0_ms'], how='outer') \
            .sort_values(['ho_type0_ms']).reset_index(drop=True).fillna(0)
        ## ul: main lost, subr safe
        table1 = dfs[0].loc[dfs[0]['lost_m'] & ~dfs[0]['lost_s'], ['ho_type0_ms']].value_counts()
        table1 = pd.DataFrame(table1).reset_index().rename(columns={0:f'ul_lost_mlss'})
        table = table.merge(table1, on=['ho_type0_ms'], how='outer') \
            .sort_values(['ho_type0_ms']).reset_index(drop=True).fillna(0)
        ## main ul excl
        table1 = dfs[0].loc[~dfs[0]['lost_m'] & dfs[0]['excl_m'], ['ho_type0_ms']].value_counts()
        table1 = pd.DataFrame(table1).reset_index().rename(columns={0:'ul_excl_m'})
        table = table.merge(table1, on=['ho_type0_ms'], how='outer') \
            .sort_values(['ho_type0_ms']).reset_index(drop=True).fillna(0)
        ## system ul excl
        # table1 = dfs[0].loc[(~dfs[0]['lost_m'] & dfs[0]['excl_m']) & (dfs[0]['excl_s']), ['ho_type0_ms']].value_counts()
        table1 = dfs[0].loc[~(dfs[0]['lost_m'] & dfs[0]['lost_s']) & dfs[0]['excl_m'] & dfs[0]['excl_s'], ['ho_type0_ms']].value_counts()
        table1 = pd.DataFrame(table1).reset_index().rename(columns={0:'ul_excl_ms'})
        table = table.merge(table1, on=['ho_type0_ms'], how='outer') \
            .sort_values(['ho_type0_ms']).reset_index(drop=True).fillna(0)
        ## ul: main excl (main not lost), subr safe (subr not lost)
        # table1 = dfs[0].loc[(~dfs[0]['lost_m'] & dfs[0]['excl_m']) & (~dfs[0]['excl_s']), ['ho_type0_ms']].value_counts()
        table1 = dfs[0].loc[~(dfs[0]['lost_m'] & dfs[0]['lost_s']) & dfs[0]['excl_m'] & ~dfs[0]['excl_s'], ['ho_type0_ms']].value_counts()
        table1 = pd.DataFrame(table1).reset_index().rename(columns={0:'ul_excl_mess'})
        table = table.merge(table1, on=['ho_type0_ms'], how='outer') \
            .sort_values(['ho_type0_ms']).reset_index(drop=True).fillna(0)
        ## ul total sent
        table1 = dfs[0].loc[:, ['ho_type0_ms']].value_counts()
        table1 = pd.DataFrame(table1).reset_index().rename(columns={0:'ul_sent'})
        table = table.merge(table1, on=['ho_type0_ms'], how='outer') \
            .sort_values(['ho_type0_ms']).reset_index(drop=True).fillna(0)
        ## ul recv
        table['ul_recv_m'] = table['ul_sent'] - table['ul_lost_m']
        table['ul_recv_ms'] = table['ul_sent'] - table['ul_lost_ms']
        # ## ul PLR
        # table['ul_PLR_m'] = (table['ul_lost_m'] / table['ul_sent'] * 100).round(2)
        # table['ul_PLR_ms'] = (table['ul_lost_ms'] / table['ul_sent'] * 100).round(2)
        # table['ul_PLR_mlss'] = (table['ul_lost_mlss'] / table['ul_sent'] * 100).round(2)
        # ## ul ELR
        # table['ul_ELR_m'] = (table['ul_excl_m'] / (table['ul_sent'] - table['ul_lost_ms']) * 100).round(2)
        # table['ul_ELR_ms'] = (table['ul_excl_ms'] / (table['ul_sent'] - table['ul_lost_ms']) * 100).round(2)
        # table['ul_ELR_mess'] = (table['ul_excl_mess'] / (table['ul_sent'] - table['ul_lost_mlss']) * 100).round(2)
    # display(table)
    
    ### Downlink
    if df_main[1].empty:
        _table = pd.DataFrame(columns=['ho_type0_ms','ho_count',
                                       'dl_lost_m','dl_lost_ms','dl_lost_mlss',
                                       'dl_excl_m','dl_excl_ms','dl_excl_mess',
                                       'dl_sent','dl_recv_m','dl_recv_ms'])
    else:
        dfs[1] = pd.merge(df_main[1], df_subr[1], on=['seq'], suffixes=('_m','_s')).copy()
        # dfs[1]['ho_type0_m'] = dfs[1]['ho_type0_m'].cat.set_categories(sorter)
        # dfs[1]['ho_type0_s'] = dfs[1]['ho_type0_s'].cat.set_categories(sorter)
        dfs[1]['ho_type0_ms'] = dfs[1]['ho_type0_m'].astype('string') + '+' + dfs[1]['ho_type0_s'].astype('string')
        dfs[1]['ho_type0_ms'] = dfs[1]['ho_type0_ms'].astype('category')
        dfs[1]['ho_type0_ms'] = dfs[1]['ho_type0_ms'].cat.set_categories(sorter1)
        ## main dl lost
        _table = dfs[1].loc[dfs[1]['lost_m'], ['ho_type0_ms']].value_counts()
        _table = pd.DataFrame(_table).reset_index().rename(columns={0:f'dl_lost_m'})
        ## system dl lost
        _table1 = dfs[1].loc[dfs[1]['lost_m'] & dfs[1]['lost_s'], ['ho_type0_ms']].value_counts()
        _table1 = pd.DataFrame(_table1).reset_index().rename(columns={0:f'dl_lost_ms'})
        _table = _table.merge(_table1, on=['ho_type0_ms'], how='outer') \
            .sort_values(['ho_type0_ms']).reset_index(drop=True).fillna(0)
        ## dl: main lost, subr safe
        _table1 = dfs[1].loc[dfs[1]['lost_m'] & ~dfs[1]['lost_s'], ['ho_type0_ms']].value_counts()
        _table1 = pd.DataFrame(_table1).reset_index().rename(columns={0:f'dl_lost_mlss'})
        _table = _table.merge(_table1, on=['ho_type0_ms'], how='outer') \
            .sort_values(['ho_type0_ms']).reset_index(drop=True).fillna(0)
        ## main dl excl
        _table1 = dfs[1].loc[~dfs[1]['lost_m'] & dfs[1]['excl_m'], ['ho_type0_ms']].value_counts()
        _table1 = pd.DataFrame(_table1).reset_index().rename(columns={0:'dl_excl_m'})
        _table = _table.merge(_table1, on=['ho_type0_ms'], how='outer') \
            .sort_values(['ho_type0_ms']).reset_index(drop=True).fillna(0)
        ## system dl excl
        # _table1 = dfs[1].loc[(~dfs[1]['lost_m'] & dfs[1]['excl_m']) & (dfs[1]['excl_s']), ['ho_type0_ms']].value_counts()
        _table1 = dfs[1].loc[~(dfs[1]['lost_m'] & dfs[1]['lost_s']) & dfs[1]['excl_m'] & dfs[1]['excl_s'], ['ho_type0_ms']].value_counts()
        _table1 = pd.DataFrame(_table1).reset_index().rename(columns={0:'dl_excl_ms'})
        _table = _table.merge(_table1, on=['ho_type0_ms'], how='outer') \
            .sort_values(['ho_type0_ms']).reset_index(drop=True).fillna(0)
        ## dl: main excl (main not lost), subr safe (subr not lost)
        # _table1 = dfs[1].loc[(~dfs[1]['lost_m'] & dfs[1]['excl_m']) & (~dfs[1]['excl_s']), ['ho_type0_ms']].value_counts()
        _table1 = dfs[1].loc[~(dfs[1]['lost_m'] & dfs[1]['lost_s']) & dfs[1]['excl_m'] & ~dfs[1]['excl_s'], ['ho_type0_ms']].value_counts()
        _table1 = pd.DataFrame(_table1).reset_index().rename(columns={0:'dl_excl_mess'})
        _table = _table.merge(_table1, on=['ho_type0_ms'], how='outer') \
            .sort_values(['ho_type0_ms']).reset_index(drop=True).fillna(0)
        ## dl total sent
        _table1 = dfs[1].loc[:, ['ho_type0_ms']].value_counts()
        _table1 = pd.DataFrame(_table1).reset_index().rename(columns={0:'dl_sent'})
        _table = _table.merge(_table1, on=['ho_type0_ms'], how='outer') \
            .sort_values(['ho_type0_ms']).reset_index(drop=True).fillna(0)
        ## dl recv
        _table['dl_recv_m'] = _table['dl_sent'] - _table['dl_lost_m']
        _table['dl_recv_ms'] = _table['dl_sent'] - _table['dl_lost_ms']
        # ## dl PLR
        # _table['dl_PLR_m'] = (_table['dl_lost_m'] / _table['dl_sent'] * 100).round(2)
        # _table['dl_PLR_ms'] = (_table['dl_lost_ms'] / _table['dl_sent'] * 100).round(2)
        # _table['dl_PLR_mlss'] = (_table['dl_lost_mlss'] / _table['dl_sent'] * 100).round(2)
        # ## dl ELR
        # _table['dl_ELR_m'] = (_table['dl_excl_m'] / (_table['dl_sent'] - _table['dl_lost_ms']) * 100).round(2)
        # _table['dl_ELR_ms'] = (_table['dl_excl_ms'] / (_table['dl_sent'] - _table['dl_lost_ms']) * 100).round(2)
        # _table['dl_ELR_mess'] = (_table['dl_excl_mess'] / (_table['dl_sent'] - _table['dl_lost_mlss']) * 100).round(2)
    # display(_table)

    ### Merge table
    table = table.merge(_table, on=['ho_type0_ms'], how='outer') \
        .sort_values(['ho_type0_ms']).reset_index(drop=True).fillna(0)
    
    ### Count HO
    df = dfs[0].copy() if not dfs[0].empty else dfs[1].copy()
    df['ho_type0_ms'] = df['ho_type0_m'].astype('string') + '+' + df['ho_type0_s'].astype('string')
    df['ho_type0_ms'] = df['ho_type0_ms'].astype('category')
    df['ho_type0_ms'] = df['ho_type0_ms'].cat.set_categories(sorter1)
    tb_tmp = df.loc[:, ['ho_type0_ms','ho_index_m','ho_index_s']].value_counts().reset_index()
    tb_dict = tb_tmp['ho_type0_ms'].value_counts().to_dict()
    table['ho_count'] = pd.Series(dtype='Int32')
    for key, val in tb_dict.items():
        table.loc[table['ho_type0_ms'] == key, 'ho_count'] = val
    # table = table.fillna(0)
    
    ## restore ho_type0_m & ho_type0_s
    table['ho_type0_m'] = [s[0] for s in table['ho_type0_ms'].str.split('+', n=1, expand=False)]
    table['ho_type0_s'] = [s[1] for s in table['ho_type0_ms'].str.split('+', n=1, expand=False)]
    table['ho_type0_m'] = table['ho_type0_m'].astype('category')
    table['ho_type0_s'] = table['ho_type0_s'].astype('category')

    ## sort by ho_type0_ms
    table['ho_type0_ms'] = table['ho_type0_ms'].cat.set_categories(sorter1)
    table = table.sort_values(['ho_type0_ms'])
    
    ## reindex
    table = table.reindex(['ho_type0_m','ho_type0_s','ho_count',
                           'ul_lost_m','ul_lost_ms','ul_lost_mlss',
                           'ul_excl_m','ul_excl_ms','ul_excl_mess',
                           'ul_sent','ul_recv_m','ul_recv_ms',
                           'dl_lost_m','dl_lost_ms','dl_lost_mlss',
                           'dl_excl_m','dl_excl_ms','dl_excl_mess',
                           'dl_sent','dl_recv_m','dl_recv_ms',
                           'ho_type0_ms'], axis=1)
    ## set dtypes
    table['ul_lost_m'] = table['ul_lost_m'].astype('Int64')
    table['ul_lost_ms'] = table['ul_lost_ms'].astype('Int64')
    table['ul_lost_mlss'] = table['ul_lost_mlss'].astype('Int64')
    table['ul_excl_m'] = table['ul_excl_m'].astype('Int64')
    table['ul_excl_ms'] = table['ul_excl_ms'].astype('Int64')
    table['ul_excl_mess'] = table['ul_excl_mess'].astype('Int64')
    table['ul_sent'] = table['ul_sent'].astype('Int64')
    table['ul_recv_m'] = table['ul_recv_m'].astype('Int64')
    table['ul_recv_ms'] = table['ul_recv_ms'].astype('Int64')
    table['dl_lost_m'] = table['dl_lost_m'].astype('Int64')
    table['dl_lost_ms'] = table['dl_lost_ms'].astype('Int64')
    table['dl_lost_mlss'] = table['dl_lost_mlss'].astype('Int64')
    table['dl_excl_m'] = table['dl_excl_m'].astype('Int64')
    table['dl_excl_ms'] = table['dl_excl_ms'].astype('Int64')
    table['dl_excl_mess'] = table['dl_excl_mess'].astype('Int64')
    table['dl_sent'] = table['dl_sent'].astype('Int64')
    table['dl_recv_m'] = table['dl_recv_m'].astype('Int64')
    table['dl_recv_ms'] = table['dl_recv_ms'].astype('Int64')
    table['ho_count'] = table['ho_count'].astype('Int32')
    
    return table

def dual_radio_merge(tb_list):
    table = tb_list[0].copy().set_index(['ho_type0_m','ho_type0_s','ho_type0_ms'])
    for i in range(1, len(tb_list)):
        table = table.add(tb_list[i].copy().set_index(['ho_type0_m','ho_type0_s','ho_type0_ms']), fill_value=0)
    table = table.reset_index()
    return table

In [22]:
table_list = []

for schm in schms:
    print(schm)
    # pprint(rrc_dict[schm])
    sorter2 = [s.split('+')[0] for s in sorter1]
    sorter3 = [s.split('+')[1] for s in sorter1]
    table = pd.DataFrame(columns=['ho_count',
                              'ul_lost_m','ul_lost_ms','ul_lost_mlss',
                              'ul_excl_m','ul_excl_ms','ul_excl_mess',
                              'ul_sent','ul_recv_m','ul_recv_ms',
                              'dl_lost_m','dl_lost_ms','dl_lost_mlss',
                              'dl_excl_m','dl_excl_ms','dl_excl_mess',
                              'dl_sent','dl_recv_m','dl_recv_ms'],
                     index=[sorter2, sorter3, sorter1]).fillna(0).reset_index() \
                    .rename(columns={'level_0':'ho_type0_m', 'level_1':'ho_type0_s', 'level_2':'ho_type0_ms'})

    for (dl1, dl2), (ul1, ul2), (ho1, ho2) in zip(dl_pkt_dict[schm], ul_pkt_dict[schm], rrc_dict[schm]):
        print(f'{ul1[:-4]}_ho.pkl')
        print(f'{dl1[:-4]}_ho.pkl')
        print(f'{ul2[:-4]}_ho.pkl')
        print(f'{dl2[:-4]}_ho.pkl')
        
        df_dl1 = set_data(pd.read_pickle(f'{dl1[:-4]}_ho.pkl'))
        df_dl2 = set_data(pd.read_pickle(f'{dl2[:-4]}_ho.pkl'))
        st = max([df_dl1['seq'].array[0], df_dl2['seq'].array[0]])
        et = min([df_dl1['seq'].array[-1], df_dl2['seq'].array[-1]])
        df_dl1 = df_dl1[(df_dl1['seq'] >= st) & (df_dl1['seq'] <= et)].reset_index(drop=True)
        df_dl2 = df_dl2[(df_dl2['seq'] >= st) & (df_dl2['seq'] <= et)].reset_index(drop=True)
        
        df_ul1 = set_data(pd.read_pickle(f'{ul1[:-4]}_ho.pkl'))
        df_ul2 = set_data(pd.read_pickle(f'{ul2[:-4]}_ho.pkl'))
        st = max([df_ul1['seq'].array[0], df_ul2['seq'].array[0]])
        et = min([df_ul1['seq'].array[-1], df_ul2['seq'].array[-1]])
        df_ul1 = df_ul1[(df_ul1['seq'] >= st) & (df_ul1['seq'] <= et)].reset_index(drop=True)
        df_ul2 = df_ul2[(df_ul2['seq'] >= st) & (df_ul2['seq'] <= et)].reset_index(drop=True)
        
        ## e.g., All+LTE
        # df_main = (pd.read_pickle(f'{ul1[:-4]}_ho.pkl'), pd.read_pickle(f'{dl1[:-4]}_ho.pkl'))
        # df_subr = (pd.read_pickle(f'{ul2[:-4]}_ho.pkl'), pd.read_pickle(f'{dl2[:-4]}_ho.pkl'))
        df_main = (df_ul1.copy(), df_dl1.copy())
        df_subr = (df_ul2.copy(), df_dl2.copy())

        _table = dual_radio_stats(df_main, df_subr)
        table = dual_radio_merge([table, _table])
        
        ## e.g., LTE+All
        # df_subr = (pd.read_pickle(f'{ul1[:-4]}_ho.pkl'), pd.read_pickle(f'{dl1[:-4]}_ho.pkl'))
        # df_main = (pd.read_pickle(f'{ul2[:-4]}_ho.pkl'), pd.read_pickle(f'{dl2[:-4]}_ho.pkl'))
        df_subr = (df_ul1.copy(), df_dl1.copy())
        df_main = (df_ul2.copy(), df_dl2.copy())

        _table = dual_radio_stats(df_main, df_subr)
        table = dual_radio_merge([table, _table])
        
        ### TODO: Add list to log tables under different schemes
        
        # break
    
    table_list.append(table)
    # break

print(len(table_list))
table_dr = dual_radio_merge(table_list)
## ul PLR
table_dr['ul_PLR_m'] = (table_dr['ul_lost_m'] / (table_dr['ul_sent'] + 1e-9) * 100).round(5)
table_dr['ul_PLR_ms'] = (table_dr['ul_lost_ms'] / (table_dr['ul_sent'] + 1e-9) * 100).round(5)
table_dr['ul_PLR_mlss'] = (table_dr['ul_lost_mlss'] / (table_dr['ul_sent'] + 1e-9) * 100).round(5)
## ul ELR
table_dr['ul_ELR_m'] = (table_dr['ul_excl_m'] / (table_dr['ul_recv_m'] + 1e-9) * 100).round(5)
table_dr['ul_ELR_ms'] = (table_dr['ul_excl_ms'] / (table_dr['ul_recv_ms'] + 1e-9) * 100).round(5)
table_dr['ul_ELR_mess'] = (table_dr['ul_excl_mess'] / (table_dr['ul_recv_ms'] + 1e-9) * 100).round(5)
## dl PLR
table_dr['dl_PLR_m'] = (table_dr['dl_lost_m'] / (table_dr['dl_sent'] + 1e-9) * 100).round(5)
table_dr['dl_PLR_ms'] = (table_dr['dl_lost_ms'] / (table_dr['dl_sent'] + 1e-9) * 100).round(5)
table_dr['dl_PLR_mlss'] = (table_dr['dl_lost_mlss'] / (table_dr['dl_sent'] + 1e-9) * 100).round(5)
## dl ELR
table_dr['dl_ELR_m'] = (table_dr['dl_excl_m'] / (table_dr['dl_recv_m'] + 1e-9) * 100).round(5)
table_dr['dl_ELR_ms'] = (table_dr['dl_excl_ms'] / (table_dr['dl_recv_ms'] + 1e-9) * 100).round(5)
table_dr['dl_ELR_mess'] = (table_dr['dl_excl_mess'] / (table_dr['dl_recv_ms'] + 1e-9) * 100).round(5)
## ul LPE
table_dr['ul_LPE_m'] = (table_dr['ul_lost_m'] / (table_dr['ho_count'] + 1e-9)).round(2)
table_dr['ul_LPE_ms'] = (table_dr['ul_lost_ms'] / (table_dr['ho_count'] + 1e-9)).round(2)
table_dr['ul_LPE_mlss'] = (table_dr['ul_lost_mlss'] / (table_dr['ho_count'] + 1e-9)).round(2)
## ul EPE
table_dr['ul_EPE_m'] = (table_dr['ul_excl_m'] / (table_dr['ho_count'] + 1e-9)).round(2)
table_dr['ul_EPE_ms'] = (table_dr['ul_excl_ms'] / (table_dr['ho_count'] + 1e-9)).round(2)
table_dr['ul_EPE_mess'] = (table_dr['ul_excl_mess'] / (table_dr['ho_count'] + 1e-9)).round(2)
## dl LPE
table_dr['dl_LPE_m'] = (table_dr['dl_lost_m'] / (table_dr['ho_count'] + 1e-9)).round(2)
table_dr['dl_LPE_ms'] = (table_dr['dl_lost_ms'] / (table_dr['ho_count'] + 1e-9)).round(2)
table_dr['dl_LPE_mlss'] = (table_dr['dl_lost_mlss'] / (table_dr['ho_count'] + 1e-9)).round(2)
## dl EPE
table_dr['dl_EPE_m'] = (table_dr['dl_excl_m'] / (table_dr['ho_count'] + 1e-9)).round(2)
table_dr['dl_EPE_ms'] = (table_dr['dl_excl_ms'] / (table_dr['ho_count'] + 1e-9)).round(2)
table_dr['dl_EPE_mess'] = (table_dr['dl_excl_mess'] / (table_dr['ho_count'] + 1e-9)).round(2)

table_dr = table_dr.reindex(['ho_type0_m','ho_type0_s','ho_count',
                             'ul_lost_m','ul_lost_ms','ul_lost_mlss',
                             'ul_excl_m','ul_excl_ms','ul_excl_mess',
                             'ul_sent','ul_recv_m','ul_recv_ms',
                             'dl_lost_m','dl_lost_ms','dl_lost_mlss',
                             'dl_excl_m','dl_excl_ms','dl_excl_mess',
                             'dl_sent','dl_recv_m','dl_recv_ms',
                             'ul_PLR_m','ul_PLR_ms','ul_PLR_mlss',
                             'ul_LPE_m','ul_LPE_ms','ul_LPE_mlss',
                             'ul_ELR_m','ul_ELR_ms','ul_ELR_mess',
                             'ul_EPE_m','ul_EPE_ms','ul_EPE_mess',
                             'dl_PLR_m','dl_PLR_ms','dl_PLR_mlss',
                             'dl_LPE_m','dl_LPE_ms','dl_LPE_mlss',
                             'dl_ELR_m','dl_ELR_ms','dl_ELR_mess',
                             'dl_EPE_m','dl_EPE_ms','dl_EPE_mess',
                             'ho_type0_ms'], axis=1)
table_dr['ho_count'] = table_dr['ho_count'].astype('Int32')
table_dr['ho_type0_m'] = table_dr['ho_type0_m'].astype('category')
table_dr['ho_type0_s'] = table_dr['ho_type0_s'].astype('category')
table_dr['ho_type0_ms'] = table_dr['ho_type0_ms'].astype('category')
table_dr['ho_type0_ms'] = table_dr['ho_type0_ms'].cat.set_categories(sorter1)
table_dr = table_dr.sort_values(['ho_type0_ms']).reset_index(drop=True)

All+LTE
/Users/jackbedford/Desktop/MOXA/Code/data/2023-04-01/Bandlock_Udp_All_LTE_B1B3_B1B8_RM500Q/qc03/#02/data/udp_uplk_loss_latency_ho.pkl
/Users/jackbedford/Desktop/MOXA/Code/data/2023-04-01/Bandlock_Udp_All_LTE_B1B3_B1B8_RM500Q/qc03/#02/data/udp_dnlk_loss_latency_ho.pkl
/Users/jackbedford/Desktop/MOXA/Code/data/2023-04-01/Bandlock_Udp_All_LTE_B1B3_B1B8_RM500Q/qc02/#01/data/udp_uplk_loss_latency_ho.pkl
/Users/jackbedford/Desktop/MOXA/Code/data/2023-04-01/Bandlock_Udp_All_LTE_B1B3_B1B8_RM500Q/qc02/#01/data/udp_dnlk_loss_latency_ho.pkl
/Users/jackbedford/Desktop/MOXA/Code/data/2023-04-01/Bandlock_Udp_All_LTE_B1B3_B1B8_RM500Q/qc03/#03/data/udp_uplk_loss_latency_ho.pkl
/Users/jackbedford/Desktop/MOXA/Code/data/2023-04-01/Bandlock_Udp_All_LTE_B1B3_B1B8_RM500Q/qc03/#03/data/udp_dnlk_loss_latency_ho.pkl
/Users/jackbedford/Desktop/MOXA/Code/data/2023-04-01/Bandlock_Udp_All_LTE_B1B3_B1B8_RM500Q/qc02/#02/data/udp_uplk_loss_latency_ho.pkl
/Users/jackbedford/Desktop/MOXA/Code/data/2023-04-01/B

In [25]:
# table_dr

In [24]:
print(len(sorter1))
print(table_dr.dtypes)
display(table_dr)
table_dr.to_pickle('table_dr.pkl')

484
ho_type0_m      category
ho_type0_s      category
ho_count           Int32
ul_lost_m          Int64
ul_lost_ms         Int64
ul_lost_mlss       Int64
ul_excl_m          Int64
ul_excl_ms         Int64
ul_excl_mess       Int64
ul_sent            Int64
ul_recv_m          Int64
ul_recv_ms         Int64
dl_lost_m          Int64
dl_lost_ms         Int64
dl_lost_mlss       Int64
dl_excl_m          Int64
dl_excl_ms         Int64
dl_excl_mess       Int64
dl_sent            Int64
dl_recv_m          Int64
dl_recv_ms         Int64
ul_PLR_m         Float64
ul_PLR_ms        Float64
ul_PLR_mlss      Float64
ul_LPE_m         Float64
ul_LPE_ms        Float64
ul_LPE_mlss      Float64
ul_ELR_m         Float64
ul_ELR_ms        Float64
ul_ELR_mess      Float64
ul_EPE_m         Float64
ul_EPE_ms        Float64
ul_EPE_mess      Float64
dl_PLR_m         Float64
dl_PLR_ms        Float64
dl_PLR_mlss      Float64
dl_LPE_m         Float64
dl_LPE_ms        Float64
dl_LPE_mlss      Float64
dl_ELR_m         Floa

,ho_type0_m,ho_type0_s,ho_count,ul_lost_m,ul_lost_ms,ul_lost_mlss,ul_excl_m,ul_excl_ms,ul_excl_mess,ul_sent,...,dl_LPE_m,dl_LPE_ms,dl_LPE_mlss,dl_ELR_m,dl_ELR_ms,dl_ELR_mess,dl_EPE_m,dl_EPE_ms,dl_EPE_mess,ho_type0_ms
0,stable,stable,1686,132629,90,132539,3050895,1229634,1953800,145470916,...,6.91,0.13,6.79,0.09063,0.00318,0.09876,54.32,1.9,59.2,stable+stable
1,stable,LTE_HO,3535,3856,2,3854,152947,8778,148023,3342739,...,0.38,0.0,0.38,0.34576,0.0465,0.44452,0.91,0.12,1.17,stable+LTE_HO
2,stable,MN_HO,13038,11416,24,11392,394394,256229,149557,9407394,...,0.52,0.0,0.52,0.32151,0.01702,0.3729,2.43,0.13,2.83,stable+MN_HO
3,stable,SN_HO,15940,17755,22,17733,320453,140689,197497,14691600,...,0.73,0.02,0.71,0.34631,0.02128,0.3706,5.35,0.33,5.73,stable+SN_HO
4,stable,MNSN_HO,0,0,0,0,0,0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,stable+MNSN_HO
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
479,NAS_Recovery_otherFailure (2),MCG_Failure_handoverFailure (1),0,0,0,0,0,0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NAS_Recovery_otherFailure (2)+MCG_Failure_hand...
480,NAS_Recovery_otherFailure (2),MCG_Failure_otherFailure (2),14,1700,6,1694,1433,285,2842,6689,...,421.14,93.79,327.36,22.54458,4.10072,61.00719,60.5,24.43,363.43,NAS_Recovery_otherFailure (2)+MCG_Failure_othe...
481,NAS_Recovery_otherFailure (2),NAS_Recovery_reconfigurationFailure (0),0,0,0,0,0,0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NAS_Recovery_otherFailure (2)+NAS_Recovery_rec...
482,NAS_Recovery_otherFailure (2),NAS_Recovery_handoverFailure (1),0,0,0,0,0,0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NAS_Recovery_otherFailure (2)+NAS_Recovery_han...


# Read records

In [3]:
import pandas as pd
import numpy as np
table_dr = pd.read_pickle('table_dr.pkl')

In [4]:
table = table_dr[['ho_type0_m','ho_type0_s','ho_count','dl_lost_ms','dl_sent','dl_PLR_ms','dl_LPE_ms']].copy()
table = table.query('ho_count > 0').reset_index(drop=True)
table.at[0, 'ho_count'] = 0
table.at[0, 'dl_LPE_ms'] = np.nan
t_lost = table.dl_lost_ms.sum()
table['dl_lost_ratio'] = table['dl_lost_ms'] / t_lost * 100
t_count = table.ho_count.sum()
table['ho_prop'] = table['ho_count'] / t_count * 100

print('Dual Radio Statistics: sort by DL lost per Event')
with pd.option_context('display.max_rows', None):
    # display(table.sort_values(by=['dl_LPE_ms'], ascending=False).query('dl_lost_ms > 0').reset_index(drop=True))
    display(table.sort_values(by=['dl_lost_ratio','dl_LPE_ms'], ascending=False).query('dl_lost_ms > 0').reset_index(drop=True))

Dual Radio Statistics: sort by DL lost per Event


,ho_type0_m,ho_type0_s,ho_count,dl_lost_ms,dl_sent,dl_PLR_ms,dl_LPE_ms,dl_lost_ratio,ho_prop
0,MCG_Failure_otherFailure (2),MCG_Failure_otherFailure (2),718,55874,792218,7.05286,77.82,32.934089,0.491666
1,SN_Setup,MCG_Failure_otherFailure (2),636,9084,197446,4.60075,14.28,5.354427,0.435515
2,MCG_Failure_otherFailure (2),SN_Setup,636,9084,197446,4.60075,14.28,5.354427,0.435515
3,SN_HO,MCG_Failure_otherFailure (2),1483,8547,1711656,0.49934,5.76,5.037901,1.015517
4,MCG_Failure_otherFailure (2),SN_HO,1483,8547,1711656,0.49934,5.76,5.037901,1.015517
5,MN_HO,MCG_Failure_otherFailure (2),1137,7846,677018,1.15891,6.9,4.624707,0.778586
6,MCG_Failure_otherFailure (2),MN_HO,1137,7846,677018,1.15891,6.9,4.624707,0.778586
7,SN_HO,SN_HO,13338,6546,32246626,0.0203,0.49,3.858441,9.133489
8,MN_HO,MN_HO,6710,5366,5178626,0.10362,0.8,3.162908,4.59482
9,MN_HO,SN_HO,6953,4574,7763965,0.05891,0.66,2.696076,4.76122


In [27]:
table = table_dr[['ho_type0_m','ho_type0_s','ho_count','ul_lost_ms','ul_sent','ul_PLR_ms','ul_LPE_ms']].copy()
table = table.query('ho_count > 0').reset_index(drop=True)
table.at[0, 'ho_count'] = 0
table.at[0, 'ul_LPE_ms'] = np.nan
t_lost = table.ul_lost_ms.sum()
table['ul_lost_ratio'] = table['ul_lost_ms'] / t_lost * 100
t_count = table.ho_count.sum()
table['ho_prop'] = table['ho_count'] / t_count * 100

print('Dual Radio Statistics: sort by UL lost per Event')
with pd.option_context('display.max_rows', None):
    # display(table.sort_values(by=['ul_LPE_ms'], ascending=False).query('ul_lost_ms > 0').reset_index(drop=True))
    display(table.sort_values(by=['ul_lost_ratio','ul_LPE_ms'], ascending=False).query('ul_lost_ms > 0').reset_index(drop=True))

Dual Radio Statistics: sort by UL lost per Event


,ho_type0_m,ho_type0_s,ho_count,ul_lost_ms,ul_sent,ul_PLR_ms,ul_LPE_ms,ul_lost_ratio,ho_prop
0,SN_Setup,NAS_Recovery_otherFailure (2),8,434,3516,12.34357,54.25,12.757202,0.005478
1,NAS_Recovery_otherFailure (2),SN_Setup,8,434,3516,12.34357,54.25,12.757202,0.005478
2,SN_Setup,SN_Setup,932,348,459262,0.07577,0.37,10.229277,0.638208
3,MN_HO,MN_HO,6710,234,3284636,0.00712,0.03,6.878307,4.59482
4,MN_HO,SN_HO,6953,109,3369412,0.00323,0.02,3.203998,4.76122
5,SN_HO,MN_HO,6953,109,3369412,0.00323,0.02,3.203998,4.76122
6,SN_Rel_MN_HO,MCG_Failure_otherFailure (2),256,105,74945,0.1401,0.41,3.08642,0.175302
7,MCG_Failure_otherFailure (2),SN_Rel_MN_HO,256,105,74945,0.1401,0.41,3.08642,0.175302
8,NAS_Recovery_otherFailure (2),NAS_Recovery_otherFailure (2),4,104,988,10.52632,26.0,3.057025,0.002739
9,MCG_Failure_otherFailure (2),MCG_Failure_otherFailure (2),718,104,910878,0.01142,0.14,3.057025,0.491666


In [28]:
ss = [
('MCG_Failure_otherFailure (2)','MCG_Failure_otherFailure (2)'),
('MCG_Failure_otherFailure (2)','SN_Setup'),
('MCG_Failure_otherFailure (2)','SN_HO'),
('MCG_Failure_otherFailure (2)','MN_HO'),
('SN_HO','SN_HO'),
('MN_HO','MN_HO'),
('MN_HO','SN_HO'),
('MCG_Failure_otherFailure (2)','stable'),
('SN_Setup','SN_Setup'),
('SN_Setup','SN_HO'),
]

for s in ss:
    print(f'{s[0]}+{s[1]}')

MCG_Failure_otherFailure (2)+MCG_Failure_otherFailure (2)
MCG_Failure_otherFailure (2)+SN_Setup
MCG_Failure_otherFailure (2)+SN_HO
MCG_Failure_otherFailure (2)+MN_HO
SN_HO+SN_HO
MN_HO+MN_HO
MN_HO+SN_HO
MCG_Failure_otherFailure (2)+stable
SN_Setup+SN_Setup
SN_Setup+SN_HO


In [29]:
ss = [
('NAS_Recovery_otherFailure (2)','SN_Setup'),
('SN_Setup','SN_Setup'),
('MN_HO','MN_HO'),
('MN_HO','SN_HO'),
('MCG_Failure_otherFailure (2)','SN_Rel_MN_HO'),
('NAS_Recovery_otherFailure (2)','NAS_Recovery_otherFailure (2)'),
('MCG_Failure_otherFailure (2)','MCG_Failure_otherFailure (2)'),
('MCG_Failure_otherFailure (2)','SN_Setup'),
('SN_Setup','stable'),
('SN_Setup','SN_Rel_MN_HO'),
]
for s in ss:
    print(f'{s[0]}+{s[1]}')

NAS_Recovery_otherFailure (2)+SN_Setup
SN_Setup+SN_Setup
MN_HO+MN_HO
MN_HO+SN_HO
MCG_Failure_otherFailure (2)+SN_Rel_MN_HO
NAS_Recovery_otherFailure (2)+NAS_Recovery_otherFailure (2)
MCG_Failure_otherFailure (2)+MCG_Failure_otherFailure (2)
MCG_Failure_otherFailure (2)+SN_Setup
SN_Setup+stable
SN_Setup+SN_Rel_MN_HO


### ranking

In [20]:
table = table_dr.copy()

In [21]:
table

,ho_type0_m,ho_type0_s,ho_count,ul_lost_m,ul_lost_ms,ul_lost_mlss,ul_excl_m,ul_excl_ms,ul_excl_mess,ul_sent,...,dl_LPE_m,dl_LPE_ms,dl_LPE_mlss,dl_ELR_m,dl_ELR_ms,dl_ELR_mess,dl_EPE_m,dl_EPE_ms,dl_EPE_mess,ho_type0_ms
0,stable,stable,1686,132629,90,132539,3050895,1229634,1953800,145470916,...,6.91,0.13,6.79,0.09063,0.00318,0.09876,54.32,1.9,59.2,stable+stable
1,stable,LTE_HO,3535,3856,2,3854,152947,8778,148023,3342739,...,0.38,0.0,0.38,0.34576,0.0465,0.44452,0.91,0.12,1.17,stable+LTE_HO
2,stable,MN_HO,13038,11416,24,11392,394394,256229,149557,9407394,...,0.52,0.0,0.52,0.32151,0.01702,0.3729,2.43,0.13,2.83,stable+MN_HO
3,stable,SN_HO,15940,17755,22,17733,320453,140689,197497,14691600,...,0.73,0.02,0.71,0.34631,0.02128,0.3706,5.35,0.33,5.73,stable+SN_HO
4,stable,MNSN_HO,0,0,0,0,0,0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,stable+MNSN_HO
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
479,NAS_Recovery_otherFailure (2),MCG_Failure_handoverFailure (1),0,0,0,0,0,0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NAS_Recovery_otherFailure (2)+MCG_Failure_hand...
480,NAS_Recovery_otherFailure (2),MCG_Failure_otherFailure (2),14,1700,6,1694,1433,285,2842,6689,...,421.14,93.79,327.36,22.54458,4.10072,61.00719,60.5,24.43,363.43,NAS_Recovery_otherFailure (2)+MCG_Failure_othe...
481,NAS_Recovery_otherFailure (2),NAS_Recovery_reconfigurationFailure (0),0,0,0,0,0,0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NAS_Recovery_otherFailure (2)+NAS_Recovery_rec...
482,NAS_Recovery_otherFailure (2),NAS_Recovery_handoverFailure (1),0,0,0,0,0,0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NAS_Recovery_otherFailure (2)+NAS_Recovery_han...


In [38]:
table = table_dr[['ho_type0_m','ho_type0_s','ho_count','dl_lost_ms','dl_sent','dl_PLR_ms','dl_LPE_ms','ho_type0_ms']].copy()

table = table.query('ho_count > 0').reset_index(drop=True)
table.at[0, 'ho_count'] = 0
table.at[0, 'dl_LPE_ms'] = np.nan

sorter = [
    'NAS_Recovery_otherFailure (2)','NAS_Recovery_handoverFailure (1)','NAS_Recovery_reconfigurationFailure (0)',
    'MCG_Failure_otherFailure (2)','MCG_Failure_handoverFailure (1)','MCG_Failure_reconfigurationFailure (0)',
    'SCG_Failure_t310-Expiry (0)','SCG_Failure_randomAccessProblem (1)','SCG_Failure_rlc-MaxNumRetx (2)','SCG_Failure_synchReconfigFailureSCG (3)',
    'SCG_Failure_scg-ReconfigFailure (4)','SCG_Failure_srb3-IntegrityFailure (5)','SCG_Failure_other-r16 (6)',
    'SN_Setup',
    'MN_HO','SN_HO','LTE_HO',
    'SN_Rel','SN_Rel_MN_HO',
    'stable',
]

table['ho_type0_m'] = table['ho_type0_m'].cat.set_categories(sorter)
table['ho_type0_s'] = table['ho_type0_s'].cat.set_categories(sorter)
table = table.sort_values(by=['ho_type0_m','ho_type0_s'], ascending=True).reset_index(drop=True)
table

indices = []

for i, row in table.iterrows():
    
    if row['ho_type0_m'] == row['ho_type0_s']:
        continue
    
    idx = table[table['ho_type0_ms'] == f'{row.ho_type0_s}+{row.ho_type0_m}'].index[0]
    if idx < i:
        continue
    
    
    indices.append(idx)

# print(indices)
table = table.drop(indices).reset_index(drop=True)

t_lost = table.dl_lost_ms.sum()
table['dl_lost_ratio'] = table['dl_lost_ms'] / t_lost * 100
t_count = table.ho_count.sum()
table['ho_prop'] = table['ho_count'] / t_count * 100

table

,ho_type0_m,ho_type0_s,ho_count,dl_lost_ms,dl_sent,dl_PLR_ms,dl_LPE_ms,ho_type0_ms,dl_lost_ratio,ho_prop
0,NAS_Recovery_otherFailure (2),NAS_Recovery_otherFailure (2),4,236,896,26.33929,59.0,NAS_Recovery_otherFailure (2)+NAS_Recovery_oth...,0.139107,0.002739
1,NAS_Recovery_otherFailure (2),MCG_Failure_otherFailure (2),14,1313,9653,13.60199,93.79,NAS_Recovery_otherFailure (2)+MCG_Failure_othe...,0.773928,0.009587
2,NAS_Recovery_otherFailure (2),SN_Setup,8,836,2858,29.25122,104.5,NAS_Recovery_otherFailure (2)+SN_Setup,0.492768,0.005478
3,NAS_Recovery_otherFailure (2),MN_HO,14,162,5984,2.70722,11.57,NAS_Recovery_otherFailure (2)+MN_HO,0.095488,0.009587
4,NAS_Recovery_otherFailure (2),SN_HO,28,859,25804,3.32894,30.68,NAS_Recovery_otherFailure (2)+SN_HO,0.506325,0.019174
...,...,...,...,...,...,...,...,...,...,...
57,SN_Rel,SN_Rel_MN_HO,11,0,211,0.0,0.0,SN_Rel+SN_Rel_MN_HO,0.0,0.007532
58,SN_Rel,stable,77,0,4057,0.0,0.0,SN_Rel+stable,0.0,0.052727
59,SN_Rel_MN_HO,SN_Rel_MN_HO,242,164,15770,1.03995,0.68,SN_Rel_MN_HO+SN_Rel_MN_HO,0.096667,0.165715
60,SN_Rel_MN_HO,stable,1330,264,193044,0.13676,0.2,SN_Rel_MN_HO+stable,0.155611,0.910747
